# Brian Dixon
## Capstone ALY6983
## March 26th, 2018

For my Capstone project I will be using data provided by UBER Movement, a database of ride information provided by UBER, to analyze the driving times from my place of work.  This area is of particular importance to me as I commute to work 5 days a weeks and I am currently in the process of looking for a new place to live.  I would like to find out what areas around Boston are most impacted by the traffic caused by Red Sox home games.  

As you may or may not know, Kenmore and the area surrounding Fenway Park is filled by an additional 40,000 people each night there is a Red Sox home game.  The extra people in the area cause traffic and congestion, which impacts a worker's commutes home. 

I hope that by doing this analysis I can determine which areas are impacted most by this excess traffic and which areas are not.  This will play an important role in determining where I may live in the future and it will be very helpful to other Bostonians who also work downtown.

I downloaded the data from the UBER Movement’s website using the criteria of  dates in which there were Red Sox homes games and dates when their were Red Sox away games.  Since UBER Movement only permitted a limit of 90 days for their date range, I needed to break up the datasets by Spring, Summer and Fall. 

Therefore, in total I had six datasets all together consisting of rider data during the weekday's PM peak driving time of 4-7PM for home games and away game for spring, summer and fall.

I used my work address, which is 1 Lincoln Street, Boston, MA, as the origin location.  From UBER Movement's website I was able to download this data as csv files.

<img src="ubermovementBoston.png" style="height:500px">

Also from the UBER Movement website I was able to download a json files containing information on all the Destination Movement ID within Massachusetts.  This json file contained the display name, Movement ID, and a list of coordinates which, when mapped, displayed a mutlipolygonal shaps outlining the Movements ID's boundries on a map.  

I will use the json file to mark the locations of all the destinations contained in the ride data.

See example of the json file below below,

In [3]:
import json
locid = json.load(open('/Users/Brian/Downloads/boston_censustracts.json'))
print(locid['features'][0]['properties'])
print(locid['features'][0]['geometry'])

{'MOVEMENT_ID': '1', 'DISPLAY_NAME': '200 Moulton Street, Rehoboth'}
{'type': 'MultiPolygon', 'coordinates': [[[[-71.29622, 41.833445], [-71.277993, 41.837346], [-71.25566, 41.845339], [-71.254675, 41.845804], [-71.238899, 41.854593], [-71.221974, 41.864], [-71.207803, 41.868011], [-71.201561, 41.869472], [-71.201136, 41.866708], [-71.193537, 41.816536], [-71.18814, 41.785424], [-71.187776, 41.783156], [-71.222663, 41.780112], [-71.283588, 41.774802], [-71.292456, 41.77488], [-71.293047, 41.784089], [-71.294799, 41.811413], [-71.29622, 41.833445]]]]}


In [19]:
# I began by importing by data sets are pandas dataframes objects 
# and labeling the data sets as either for being a Red Sox home game or an away game.

import numpy as np
import pandas as pd
Spring_Home = pd.read_csv('Travel_Times_Spring_Home.csv')
Summer_Home = pd.read_csv('Travel_Times_Summer_Home.csv')
Fall_Home = pd.read_csv('Travel_Times_Fall_Home.csv')
Spring_Away = pd.read_csv('Travel_Times_Spring_Away.csv')
Summer_Away = pd.read_csv('Travel_Times_Summer_Away.csv')
Fall_Away = pd.read_csv('Travel_Times_Fall_Away.csv')

Spring_Home['Red Sox Game'] = 'Home'
Summer_Home['Red Sox Game'] = 'Home'
Fall_Home['Red Sox Game'] = 'Home'

Spring_Away['Red Sox Game'] = 'Away'
Summer_Away['Red Sox Game'] = 'Away'
Fall_Away['Red Sox Game'] = 'Away'

Fall_Away.head()

,Origin Movement ID,Origin Display Name,Destination Movement ID,Destination Display Name,Date Range,Mean Travel Time (Seconds),Range - Lower Bound Travel Time (Seconds),Range - Upper Bound Travel Time (Seconds),Red Sox Game
0,885,"1 Lincoln St, Boston, MA",3,"0 Flaherty Way, D Street / West Broadway, Boston","8/28/2017 - 9/24/2017, Weekdays, PM Peak",641,373,1101,Away
1,885,"1 Lincoln St, Boston, MA",5,"0 Queensberry Street, Fenway/Kenmore, Boston","8/28/2017 - 9/24/2017, Weekdays, PM Peak",1152,827,1604,Away
2,885,"1 Lincoln St, Boston, MA",13,"200 Liberty Street, Braintree","8/28/2017 - 9/24/2017, Weekdays, PM Peak",2009,1402,2878,Away
3,885,"1 Lincoln St, Boston, MA",14,"400 Cushing Street, Hingham","8/28/2017 - 9/24/2017, Weekdays, PM Peak",2442,1741,3423,Away
4,885,"1 Lincoln St, Boston, MA",16,"100 Kenrick Street, Oak Square, Boston","8/28/2017 - 9/24/2017, Weekdays, PM Peak",1734,1236,2432,Away


In [20]:
#Next, I merged the data set for each season into a single data from for home games and away games
# using the merge function.  The common point I used was the Destination Movement ID.

#To merge to one data fame from home games
Traffic_Home_Games = pd.merge(Spring_Home, Summer_Home, left_on = 'Destination Movement ID', right_on = 'Destination Movement ID', how = 'inner')
Traffic_Home_Games = pd.merge(Traffic_Home_Games, Fall_Home, left_on = 'Destination Movement ID', right_on = 'Destination Movement ID', how = 'inner')


#To merge to one data fame from away games
Traffic_Away_Games = pd.merge(Spring_Away, Summer_Away, left_on = 'Destination Movement ID', right_on = 'Destination Movement ID', how = 'inner')
Traffic_Away_Games = pd.merge(Traffic_Away_Games, Fall_Away, left_on = 'Destination Movement ID', right_on = 'Destination Movement ID', how = 'inner')

Traffic_Home_Games.head()



,Origin Movement ID_x,Origin Display Name_x,Destination Movement ID,Destination Display Name_x,Date Range_x,Mean Travel Time (Seconds)_x,Range - Lower Bound Travel Time (Seconds)_x,Range - Upper Bound Travel Time (Seconds)_x,Red Sox Game_x,Origin Movement ID_y,...,Range - Upper Bound Travel Time (Seconds)_y,Red Sox Game_y,Origin Movement ID,Origin Display Name,Destination Display Name,Date Range,Mean Travel Time (Seconds),Range - Lower Bound Travel Time (Seconds),Range - Upper Bound Travel Time (Seconds),Red Sox Game
0,885,"1 Lincoln St, Boston, MA",3,"0 Flaherty Way, D Street / West Broadway, Boston","4/3/2017 - 5/27/2017, Weekdays, PM Peak",670,391,1145,Home,885,...,1131,Home,885,"1 Lincoln St, Boston, MA","0 Flaherty Way, D Street / West Broadway, Boston","8/27/2017 - 10/1/2017, Weekdays, PM Peak",692,398,1202,Home
1,885,"1 Lincoln St, Boston, MA",5,"0 Queensberry Street, Fenway/Kenmore, Boston","4/3/2017 - 5/27/2017, Weekdays, PM Peak",1236,907,1683,Home,885,...,1776,Home,885,"1 Lincoln St, Boston, MA","0 Queensberry Street, Fenway/Kenmore, Boston","8/27/2017 - 10/1/2017, Weekdays, PM Peak",1318,961,1807,Home
2,885,"1 Lincoln St, Boston, MA",13,"200 Liberty Street, Braintree","4/3/2017 - 5/27/2017, Weekdays, PM Peak",1829,1208,2768,Home,885,...,3179,Home,885,"1 Lincoln St, Boston, MA","200 Liberty Street, Braintree","8/27/2017 - 10/1/2017, Weekdays, PM Peak",1946,1321,2865,Home
3,885,"1 Lincoln St, Boston, MA",14,"400 Cushing Street, Hingham","4/3/2017 - 5/27/2017, Weekdays, PM Peak",2352,1628,3396,Home,885,...,3557,Home,885,"1 Lincoln St, Boston, MA","400 Cushing Street, Hingham","8/27/2017 - 10/1/2017, Weekdays, PM Peak",2297,1557,3387,Home
4,885,"1 Lincoln St, Boston, MA",16,"100 Kenrick Street, Oak Square, Boston","4/3/2017 - 5/27/2017, Weekdays, PM Peak",1727,1273,2342,Home,885,...,2405,Home,885,"1 Lincoln St, Boston, MA","100 Kenrick Street, Oak Square, Boston","8/27/2017 - 10/1/2017, Weekdays, PM Peak",1776,1323,2382,Home


In [21]:
#From here I took the Mean Travel Time for each season's dataset and calcumated the average.
#I assinged this number to a new coumn called 'Mean_Time'
Traffic_Home_Games['Mean_Time'] = ((Traffic_Home_Games['Mean Travel Time (Seconds)_x'] + Traffic_Home_Games['Mean Travel Time (Seconds)_y'] + Traffic_Home_Games['Mean Travel Time (Seconds)'] )/3)
Traffic_Away_Games['Mean_Time'] = ((Traffic_Away_Games['Mean Travel Time (Seconds)_x'] + Traffic_Away_Games['Mean Travel Time (Seconds)_y'] + Traffic_Away_Games['Mean Travel Time (Seconds)'] )/3)


Traffic_Away_Games['Mean_Time'].head()

0     665.333333
1    1139.333333
2    2010.000000
3    2406.666667
4    1683.333333
Name: Mean_Time, dtype: float64

In [22]:
#Next I cleaned up the dataframes by only keeping the columns I need.
Traffic_Away_Games = Traffic_Away_Games.iloc[:,[1,2,3, 8, 25]]
Traffic_Home_Games = Traffic_Home_Games.iloc[:,[1,2,3, 8, 25]]
Traffic_Home_Games.head()

,Origin Display Name_x,Destination Movement ID,Destination Display Name_x,Red Sox Game_x,Mean_Time
0,"1 Lincoln St, Boston, MA",3,"0 Flaherty Way, D Street / West Broadway, Boston",Home,669.333333
1,"1 Lincoln St, Boston, MA",5,"0 Queensberry Street, Fenway/Kenmore, Boston",Home,1284.000000
2,"1 Lincoln St, Boston, MA",13,"200 Liberty Street, Braintree",Home,1997.666667
3,"1 Lincoln St, Boston, MA",14,"400 Cushing Street, Hingham",Home,2419.333333
4,"1 Lincoln St, Boston, MA",16,"100 Kenrick Street, Oak Square, Boston",Home,1739.333333


In [9]:
#From here I was able to create a dataframe using the data from the boston_censustracts.json file taking the average lattitude
# and longitude for each of the Movement ID's coordinates.

import json
locid = json.load(open('/Users/Brian/Downloads/boston_censustracts.json'))


lat = []
lon = []
move_id = []
desc_id = []
j = 0
while j < len(locid['features']):
    lats = []
    lons = []
    i = 0
    while i < len(locid['features'][j]['geometry']['coordinates'][0][0]):
        lats.append(locid['features'][j]['geometry']['coordinates'][0][0][i][1])
        lons.append(locid['features'][j]['geometry']['coordinates'][0][0][i][0])
        i = i + 1
    lat.append(np.mean(lats))
    lon.append(np.mean(lons))
    desc_id.append(locid['features'][j]['properties']['DISPLAY_NAME'])
    move_id.append(int(locid['features'][j]['properties']['MOVEMENT_ID']))
    j = j +1


cord_df = pd.DataFrame(
    {'lat': lat,
     'lon': lon,
     'move_id': move_id,
     'desc_id': desc_id
    })        
cord_df.head()   

,desc_id,lat,lon,move_id
0,"200 Moulton Street, Rehoboth",41.823810,-71.244897,1
1,"Academic Commons at Goddard Library, Universit...",42.251725,-71.823422,2
2,"0 Flaherty Way, D Street / West Broadway, Boston",42.338649,-71.054129,3
3,"0 Moulton Street, Lynn",42.467473,-70.970194,4
4,"0 Queensberry Street, Fenway/Kenmore, Boston",42.343344,-71.096114,5


In [13]:
#Next I was able to using the geopy library and use the distance fuction to calcute the distance, in miles, for each Movement ID
# from my work address of 1 Lincoln Street.  
#Before starting this, I found that the lon/lat coordiantes of my work are 42.3526844 N, -71.0583632 W.

import geopy.distance
i = 0
distance = []
while i < len(cord_df):
    lat_calc = cord_df.loc[i, "lat"]
    lon_calc = cord_df.loc[i, "lon"]
    coords_dest = (lat_calc, lon_calc)
    #1 Lincoln
    coords_origin = (42.3526844, -71.0583632)
    coords_origin_fenway = (42.346268, -71.095764)
    distance.append(geopy.distance.vincenty(coords_dest, coords_origin).miles)
    i = i + 1
cord_df['Distance from Work'] = distance
cord_df.head()

,desc_id,lat,lon,move_id,Distance from Work
0,"200 Moulton Street, Rehoboth",41.823810,-71.244897,1,37.740926
1,"Academic Commons at Goddard Library, Universit...",42.251725,-71.823422,2,39.813543
2,"0 Flaherty Way, D Street / West Broadway, Boston",42.338649,-71.054129,3,0.992746
3,"0 Moulton Street, Lynn",42.467473,-70.970194,4,9.116570
4,"0 Queensberry Street, Fenway/Kenmore, Boston",42.343344,-71.096114,5,2.037529


In [23]:
#From here I merged the two dataframes using the Destination Movement ID and move_id
# I will now have two data frames of the mean travel times to various places along with their latitude and longitude
# as well as that points distance from my work.

Traffic_Away_Games = pd.merge(Traffic_Away_Games, cord_df, left_on = 'Destination Movement ID', right_on = 'move_id', how = 'inner')
Traffic_Home_Games = pd.merge(Traffic_Home_Games, cord_df, left_on = 'Destination Movement ID', right_on = 'move_id', how = 'inner')

Traffic_Away_Games.head()

,Origin Display Name_x,Destination Movement ID,Destination Display Name_x,Red Sox Game_x,Mean_Time,desc_id,lat,lon,move_id,Distance from Work
0,"1 Lincoln St, Boston, MA",3,"0 Flaherty Way, D Street / West Broadway, Boston",Away,665.333333,"0 Flaherty Way, D Street / West Broadway, Boston",42.338649,-71.054129,3,0.992746
1,"1 Lincoln St, Boston, MA",5,"0 Queensberry Street, Fenway/Kenmore, Boston",Away,1139.333333,"0 Queensberry Street, Fenway/Kenmore, Boston",42.343344,-71.096114,5,2.037529
2,"1 Lincoln St, Boston, MA",13,"200 Liberty Street, Braintree",Away,2010.000000,"200 Liberty Street, Braintree",42.214907,-70.980800,13,10.306984
3,"1 Lincoln St, Boston, MA",14,"400 Cushing Street, Hingham",Away,2406.666667,"400 Cushing Street, Hingham",42.204864,-70.885154,14,13.524580
4,"1 Lincoln St, Boston, MA",16,"100 Kenrick Street, Oak Square, Boston",Away,1683.333333,"100 Kenrick Street, Oak Square, Boston",42.347414,-71.167091,16,5.578538


In [24]:
#The dataset had destination points from all over Massachusetts,
#however I was only intersted in information pertain to only what I coniser the 'Boston Area'
#since this was the area I will eventually plan to move to
#so I decided to only from destination points for area that are within 8 miles from my work 

Traffic_Home_Games = Traffic_Home_Games.loc[Traffic_Home_Games['Distance from Work']< 8]
Traffic_Away_Games = Traffic_Away_Games.loc[Traffic_Away_Games['Distance from Work']< 8]
Traffic_Away_Games.head()

,Origin Display Name_x,Destination Movement ID,Destination Display Name_x,Red Sox Game_x,Mean_Time,desc_id,lat,lon,move_id,Distance from Work
0,"1 Lincoln St, Boston, MA",3,"0 Flaherty Way, D Street / West Broadway, Boston",Away,665.333333,"0 Flaherty Way, D Street / West Broadway, Boston",42.338649,-71.054129,3,0.992746
1,"1 Lincoln St, Boston, MA",5,"0 Queensberry Street, Fenway/Kenmore, Boston",Away,1139.333333,"0 Queensberry Street, Fenway/Kenmore, Boston",42.343344,-71.096114,5,2.037529
4,"1 Lincoln St, Boston, MA",16,"100 Kenrick Street, Oak Square, Boston",Away,1683.333333,"100 Kenrick Street, Oak Square, Boston",42.347414,-71.167091,16,5.578538
5,"1 Lincoln St, Boston, MA",17,"0 Maggi Road, Revere",Away,1666.000000,"0 Maggi Road, Revere",42.421630,-71.001318,17,5.582633
7,"1 Lincoln St, Boston, MA",20,"0 Florence Street, Everett",Away,1656.666667,"0 Florence Street, Everett",42.405084,-71.043400,20,3.696908


In [36]:
#Below I created a new dataframe which contains data from home and away games
#I created a new column calculating the difference in mean times between home and away games.
#If the difference was larger than 1.5 times the average difference
#then I labled these as locations in which there is Fenway Traffic.

Traffic_Baseball = pd.merge(Traffic_Home_Games, Traffic_Away_Games, left_on = 'Destination Movement ID', right_on = 'Destination Movement ID', how = 'inner')
Traffic_Baseball["Difference"] = Traffic_Baseball['Mean_Time_x'] - Traffic_Baseball['Mean_Time_y']
Traffic_Baseball["Baseball"] = "No Fenway Traffic"
Traffic_Baseball.loc[Traffic_Baseball["Difference"] > 1.5 * np.mean(Traffic_Baseball["Difference"]), ["Baseball"]] = "Fenway Traffic"

Traffic_Baseball.head()

,Origin Display Name_x_x,Destination Movement ID,Destination Display Name_x_x,Red Sox Game_x_x,Mean_Time_x,desc_id_x,lat_x,lon_x,move_id_x,Distance from Work_x,...,Destination Display Name_x_y,Red Sox Game_x_y,Mean_Time_y,desc_id_y,lat_y,lon_y,move_id_y,Distance from Work_y,Difference,Baseball
0,"1 Lincoln St, Boston, MA",3,"0 Flaherty Way, D Street / West Broadway, Boston",Home,669.333333,"0 Flaherty Way, D Street / West Broadway, Boston",42.338649,-71.054129,3,0.992746,...,"0 Flaherty Way, D Street / West Broadway, Boston",Away,665.333333,"0 Flaherty Way, D Street / West Broadway, Boston",42.338649,-71.054129,3,0.992746,4.000000,No Fenway Traffic
1,"1 Lincoln St, Boston, MA",5,"0 Queensberry Street, Fenway/Kenmore, Boston",Home,1284.000000,"0 Queensberry Street, Fenway/Kenmore, Boston",42.343344,-71.096114,5,2.037529,...,"0 Queensberry Street, Fenway/Kenmore, Boston",Away,1139.333333,"0 Queensberry Street, Fenway/Kenmore, Boston",42.343344,-71.096114,5,2.037529,144.666667,Fenway Traffic
2,"1 Lincoln St, Boston, MA",16,"100 Kenrick Street, Oak Square, Boston",Home,1739.333333,"100 Kenrick Street, Oak Square, Boston",42.347414,-71.167091,16,5.578538,...,"100 Kenrick Street, Oak Square, Boston",Away,1683.333333,"100 Kenrick Street, Oak Square, Boston",42.347414,-71.167091,16,5.578538,56.000000,Fenway Traffic
3,"1 Lincoln St, Boston, MA",17,"0 Maggi Road, Revere",Home,1548.333333,"0 Maggi Road, Revere",42.421630,-71.001318,17,5.582633,...,"0 Maggi Road, Revere",Away,1666.000000,"0 Maggi Road, Revere",42.421630,-71.001318,17,5.582633,-117.666667,No Fenway Traffic
4,"1 Lincoln St, Boston, MA",20,"0 Florence Street, Everett",Home,1620.000000,"0 Florence Street, Everett",42.405084,-71.043400,20,3.696908,...,"0 Florence Street, Everett",Away,1656.666667,"0 Florence Street, Everett",42.405084,-71.043400,20,3.696908,-36.666667,No Fenway Traffic


In [40]:
#Below is a scatter plot showing the differnce in mean travel time to various destinations
#for home games versus away games

from bokeh.plotting import figure, output_file, show
from bokeh.charts import Line
from bokeh.io import output_notebook
output_notebook()

x = Traffic_Baseball['Distance from Work_x']
y = Traffic_Baseball['Mean_Time_x']
x2 = Traffic_Baseball['Distance from Work_x']
y2 = Traffic_Baseball['Mean_Time_y']
z = Traffic_Baseball['Difference']/10




good = figure(x_axis_label="Distance from Work", y_axis_label="Mean Travel Time (Seconds)")
good.circle(
    x, y, size=abs(z),
    color='red')
good.circle(
    x2, y2, size=abs(z),
    color='grey')
home_mph = np.mean(Traffic_Home_Games['Distance from Work'])/(np.mean(Traffic_Home_Games['Mean_Time'])/3600)
away_mph = np.mean(Traffic_Away_Games['Distance from Work'])/(np.mean(Traffic_Away_Games['Mean_Time'])/3600)
print ("The average speed for an Uber driver during an Away Game is ", round(away_mph,2), "mph")
print ("The average speed for an Uber driver during a Home Game is ", round(home_mph,2), "mph")

show(good)

Loading BokehJS ...

The average speed for an Uber driver during an Away Game is  10.2 mph
The average speed for an Uber driver during a Home Game is  10.01 mph


In [42]:
#Below will be the plotted points on an embeeded google map's map showing the location in which there is Fenway traffic is red
# and location in which there is no Fenway traffic in Blue

import gmplot

gmapb = gmplot.GoogleMapPlotter(42.3526844, -71.0583632, 12)
gmapb.scatter(Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "No Fenway Traffic", ['lat_y']].values, Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "No Fenway Traffic", ['lon_y']].values, 'blue', size=100, marker=False)
gmapb.scatter(Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "Fenway Traffic", ['lat_y']].values, Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "Fenway Traffic", ['lon_y']].values, 'red', size=100, marker=False)

gmapb.draw("mymap.html")


In [211]:
#Below in a Knn Classifier algoriths that classifiers using the 'lat' and 'lon' columns.
#What is printed is a matrix indicated the predictions were correct 82% of the time
from sklearn.neighbors import KNeighborsClassifier

Traffic_Baseball_Train = Traffic_Baseball.iloc[0:245, :]
Traffic_Baseball_Test = Traffic_Baseball.iloc[246:346, :]

neigh = KNeighborsClassifier(n_neighbors=21)
neigh.fit(Traffic_Baseball_Train.iloc[:, 6:8], Traffic_Baseball_Train['Baseball'])
predict = neigh.predict(Traffic_Baseball_Test.iloc[:, 6:8]).tolist()

results = pd.DataFrame({"Location": Traffic_Baseball_Test['Destination Movement ID'], "Actual": Traffic_Baseball_Test['Baseball'], "Predictions": predict, "Correct": Traffic_Baseball_Test['Baseball']== predict})
results.head()
print(results['Correct'].value_counts())


True     82
False    18
Name: Correct, dtype: int64


In [212]:
#Below is a new map which which has points for the point with and without traffic
#As well as new point overlayed with the prediction
#Is the points are solid blue or red, then the prediction are correct
#If the point is blue with a red center, then the location has Fenway traffic, but the classifier predicted it did not.
#If the point is red with a blue center, then the location does not have Fenway traffic, but the classifier predicted it does.

predict_all = neigh.predict(Traffic_Baseball.iloc[:, 6:8]).tolist()

results_all = pd.DataFrame({"Location": Traffic_Baseball['Destination Movement ID'], "Actual": Traffic_Baseball['Baseball'], "Predictions": predict_all, "Correct": Traffic_Baseball['Baseball']== predict_all})
results.head()

Traffic_Baseball_With_Predictions = pd.merge(Traffic_Baseball, results_all, left_on = 'Destination Movement ID', right_on = 'Location', how = 'inner')

gmapb2 = gmplot.GoogleMapPlotter(42.3526844, -71.0583632, 12)
gmapb2.scatter(Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "No Fenway Traffic", ['lat_y']].values, Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "No Fenway Traffic", ['lon_y']].values, 'blue', size=100, marker=False)
gmapb2.scatter(Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "Fenway Traffic", ['lat_y']].values, Traffic_Baseball.loc[Traffic_Baseball['Baseball'] == "Fenway Traffic", ['lon_y']].values, 'red', size=100, marker=False)
gmapb2.scatter(Traffic_Baseball_With_Predictions.loc[Traffic_Baseball_With_Predictions['Predictions'] == "No Fenway Traffic", ['lat_y']].values, Traffic_Baseball_With_Predictions.loc[Traffic_Baseball_With_Predictions['Predictions'] == "No Fenway Traffic", ['lon_y']].values, 'blue', size=200, marker=False)
gmapb2.scatter(Traffic_Baseball_With_Predictions.loc[Traffic_Baseball_With_Predictions['Predictions'] == "Fenway Traffic", ['lat_y']].values, Traffic_Baseball_With_Predictions.loc[Traffic_Baseball_With_Predictions['Predictions'] == "Fenway Traffic", ['lon_y']].values, 'red', size=200, marker=False)
gmapb2.draw("mymap2.html")

In [214]:
#Lastly, I built a function which takes as an input a single address
#and returns as to whether or not the commute would experience Fenway Traffic
#if leaving from my work Monday to Friday 4:00-7:00 PM to that address
#during a Red Sox home game.


#**WARNING: If you get "GeocoderTimedOut: Service timed out", you can wait a few seconds and try again, and it should work.**

location = geolocator.geocode("1, Pauline Street, Winthrop, Suffolk County, Massachusetts, 02152")  

from geopy.geocoders import Nominatim
geolocator = Nominatim()
def fenway_traffic(x):
    address = x
    prediction = neigh.predict(address.iloc[:, 0:]).tolist()
    return(prediction[0])

 
address = (location.latitude, location.longitude)
latlon = np.array([address[0], address[1]])
df = pd.DataFrame(latlon.reshape(1,2), columns = list("ab"))

print(fenway_traffic(df))


No Fenway Traffic
